# VDF Function Search For edges

In order to apply any system optimal route assignment algorithms, volume delay functions for each edge in the graph has to be defined. This will be estimated from volume travel time pairs recorded from 4 simulations runs, each with 20 iterations. The travel volume was scaled from 10% to 100% at a step of 10% per iteration, so the the final 10 iterations of each run are all at 100% volume.

In [7]:
import qgrid
import pandas as pd
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from typing import Sequence

# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

## Load and display the data

In [2]:
edge_volume_time = pd.read_pickle(r"..\Urban Freeway Dyn Assign Redmond.US\edgeVolTime.pk1")
edge_attr = pd.read_pickle(r"..\Urban Freeway Dyn Assign Redmond.US\edges_attr.pk1")

### Dynamic Assignment Edge Attributes

In [3]:
qgrid.show_grid(edge_attr, grid_options={'editable': False,'maxVisibleRows':8})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Simulation results

|     NO     |   VehType    |              Period                         |        TRAVTMNEW         |       VOLNEW        | ITR
|:----------:|:------------:|:-------------------------------------------:|:------------------------:|:------------------:|:-----------------:|
|edge number | vehicle type | simulation data sample period (15 min each) | average edge travel time | 15 min volume count | simulation iteration |


In [4]:
qgrid.show_grid(edge_volume_time, grid_options={'editable': False,'maxVisibleRows':8})

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Create interactive travel time vs 15 min volume plot

In [15]:
def plot_VD(edge: int, veh_type: str, period: Sequence[int]):

    # Filter for plotting
    plotting_frame = edge_volume_time.loc[(edge_volume_time['NO'] == edge) &
                                          (edge_volume_time['VehType'] == veh_type) &
                                          (edge_volume_time['Period'] >= period[0]) &
                                          (edge_volume_time['Period'] <= period[1]), :]

    # get different iteration series for plotly
    data = [go.Scatter(
        x=plotting_frame.VOLNEW,
        y=plotting_frame.TRAVTMNEW,
        mode='markers',
        marker=dict(
            color=plotting_frame.ITR,
            colorbar=dict(title='Sim Iteration'),
            colorscale='Bluered'
        )
    )]

    fig = {
        'data': data,
        'layout': {
            'xaxis': {'title': '15 min Volume (count)', 'rangemode': 'tozero'},
            'yaxis': {'title': "Average Vehicle Travel Time (s)", 'rangemode': 'tozero'},
            #'template': 'plotly_dark'
        }
    }

    iplot(fig)  # print plot

In [18]:
# Create interactive plot
# Build widgets
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')
edge_wig = widgets.IntSlider(min=edge_attr.index.min(),
                             max=edge_attr.index.max(),
                             step=1,
                             description='Network Edge:',
                             value=75,
                             style=style,
                             layout=layout)
veh_type_wig = widgets.Dropdown(options=list(edge_volume_time['VehType'].unique()),
                                description='Vehicle Type:',
                                value='ALL',
                                style=style)
period_wig = widgets.IntRangeSlider(min=edge_volume_time['Period'].min(),
                                        max=edge_volume_time['Period'].max(),
                                        step=1,
                                        description='Data period:',
                                        style=style,
                                        layout=layout,
                                        value=(2,4))
# Final plot
interact(plot_VD, edge=edge_wig, veh_type=veh_type_wig,
         period=period_wig);

interactive(children=(IntSlider(value=75, description='Network Edge:', layout=Layout(width='95%'), max=1327, m…